In [1]:
import pandas as pd
import numpy as np
import glob
import lightgbm as lgb

data_path = r'F:\contest\ATEC\dataset'
data_files_paths = glob.glob(data_path + '\*.csv')
#data_files_paths



In [2]:
data_reader = pd.read_csv(data_files_paths[-1], iterator=True)
traindata = {}
for i in range(5):
    traindata[i] = data_reader.get_chunk(200000)
    # 因为疏忽写文件时将index保留了，但是在读时应该去掉
    print('[+] get data {} is ok'.format(i))
    

[+] get data 0 is ok
[+] get data 1 is ok
[+] get data 2 is ok
[+] get data 3 is ok
[+] get data 4 is ok


In [11]:
np.isnan(traindata[0].f234).sum()[0]

IndexError: invalid index to scalar variable.

In [3]:

    
cols = list(traindata[0].columns)
cols.remove('id')
#cols.remove('Unnamed: 0')
cols.remove('label')
cols.remove('date')

nan_col = []
for i in cols:
    column_fea = pd.DataFrame()
    for j in range(5):
        # 将某一特征全部数据合并
        column = traindata[j][[i, 'label']].copy()
        column_fea = pd.concat([column_fea, column])
    if np.isnan(column_fea[[i]]).sum()[0] > 0:
        print('[{}] nan is {}'.format(i, np.isnan(column_fea[[i]]).sum()[0]))
        if np.isnan(column_fea[[i]]).sum()[0] > 200000:
            nan_col.append(i)
            
for i in nan_col:
    cols.remove(i)
    
train_fir = pd.DataFrame()
for i in range(5):
    train_mid = traindata[i][cols + ['label']].copy()
    train_fir = pd.concat([train_fir, train_mid])
    print('[+] concating data {}'.format(i), 'is ok')
    
    


[f5] nan is 199825
[f20] nan is 207448
[f21] nan is 207448
[f22] nan is 207448
[f23] nan is 207448
[f24] nan is 207585
[f25] nan is 207585
[f26] nan is 207585
[f27] nan is 207585
[f28] nan is 211036
[f29] nan is 211036
[f30] nan is 211036
[f31] nan is 211036
[f32] nan is 341887
[f33] nan is 341887
[f34] nan is 341887
[f35] nan is 341887
[f36] nan is 925781
[f37] nan is 925781
[f38] nan is 925781
[f39] nan is 925781
[f40] nan is 925781
[f41] nan is 925781
[f42] nan is 925781
[f43] nan is 925781
[f44] nan is 925781
[f45] nan is 925781
[f46] nan is 925781
[f47] nan is 925781
[f48] nan is 341661
[f49] nan is 341661
[f50] nan is 341661
[f51] nan is 341661
[f52] nan is 207448
[f53] nan is 207448
[f54] nan is 341661
[f55] nan is 341661
[f56] nan is 341661
[f57] nan is 341661
[f58] nan is 341661
[f59] nan is 341661
[f60] nan is 341661
[f61] nan is 341661
[f62] nan is 341661
[f63] nan is 341661
[f64] nan is 270515
[f65] nan is 270515
[f66] nan is 270515
[f67] nan is 270515
[f68] nan is 270515
[

In [17]:
test = pd.read_csv(data_files_paths[0])

161

In [4]:
lg = lgb.LGBMClassifier(max_depth=10, min_child_samples=100)
tra = train_fir.iloc[0:700000]
tes = train_fir.iloc[700000:]
lg.fit(tra[cols], tra['label'])

# test = pd.read_csv(data_files_paths[0])
# out = lg.predict_proba(test[cols])
# sub = test[['id']].copy()
# sub['score'] = out[:, 1]
# sub.to_csv('submission_1.csv', index=False)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        learning_rate=0.1, max_depth=10, min_child_samples=100,
        min_child_weight=0.001, min_split_gain=0.0, n_estimators=100,
        n_jobs=-1, num_leaves=31, objective=None, random_state=None,
        reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=1.0,
        subsample_for_bin=200000, subsample_freq=1)

In [28]:
res = lg.predict_proba(tes[cols])

In [33]:
from sklearn.metrics import roc_curve 
y = tes['label']
pred = res[:,2]
def score(y,pred): 
    fpr, tpr, thresholds = roc_curve(y, pred, pos_label=1) 
    score=0.4*tpr[np.where(fpr>=0.001)[0][0]]+0.3*tpr[np.where(fpr>=0.005)[0][0]]+0.3*tpr[np.where(fpr>=0.01)[0][0]] 
    return score 
score(y,pred)

0.413830376940133

1293